## Imports

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose

# TensorFlow Baby
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import Dense, Input # type: ignore
from tensorflow.keras.losses import MeanAbsoluteError # type: ignore
from tensorflow.keras.callbacks  import ModelCheckpoint # type: ignore

# Chart drawing
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(255,255,255,255)', plot_bgcolor='rgba(0,0,0,0.8)')

fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)

pio.templates['xgboost'] = templated_fig.layout.template
pio.templates.default = 'xgboost'


## Data Loader

In [ ]:
ETF_NAME = 'ADCORP'
ETF_DIRECTORY = r'../data/csv/bidcorp.csv'

df = pd.read_csv(os.path.join(ETF_DIRECTORY))

df['Date'] = pd.to_datetime(df['Date'])

df = df[(df['Date'].dt.year >= 2020)].copy()

df.index = range(len(df))

df.rename(columns={'Closing (c)': 'Close'}, inplace=True)
df.rename(columns={'High (c)': 'High'}, inplace=True)
df.rename(columns={'Low (c)': 'Low'}, inplace=True)

df = df.sort_values(by='Date')

## Decomposers

In [ ]:
df_close = df[['Date', 'Close']].copy()
df_close = df_close.set_index('Date')
df_close.head()

decomp = seasonal_decompose(df_close, period=365)
fig = decomp.plot()
fig.set_size_inches(20, 8)

## Technical indicators

### Moving Averages


In [ ]:
df['EMA_9'] = df['Close'].ewm(9).mean().shift()
df['SMA_5'] = df['Close'].rolling(5).mean().shift()
df['SMA_10'] = df['Close'].rolling(10).mean().shift()
df['SMA_15'] = df['Close'].rolling(15).mean().shift()
df['SMA_30'] = df['Close'].rolling(30).mean().shift()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date, y=df.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_5, name='SMA 5'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_10, name='SMA 10'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_15, name='SMA 15'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_30, name='SMA 30'))
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close', opacity=0.2))
fig.show()

### Relative Strength Index (RSI)

In [ ]:
def relative_strength_idx(df, n=14):
    close = df['Close']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

df['RSI'] = relative_strength_idx(df)

fig = go.Figure(go.Scatter(x=df.Date, y=df.RSI, name='RSI'))
fig.show()

### MACD

In [ ]:
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())

df['MACD'] = pd.Series(EMA_12 - EMA_26)
df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_12, name='EMA 12'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_26, name='EMA 26'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=df['MACD'], name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=df['MACD_signal'], name='Signal line'), row=2, col=1)
fig.show()

### Shift label column

Because I want to predict the next day price, after calculating all features for day I shift Close price column by -1 rows. After doing that, for day  Di we have features from the same timestamp e.g.  RSI  but the price  Ci+1 from upcoming day.

In [ ]:
df['Close'] = df['Close'].shift(-1)


### Drop invalid samples

Because of calculating moving averages and shifting label column, few rows will have invalid values i.e. we haven't calculated  SMA10 for the first 10 days. Moreover, after shifting Close price column, last row price is equal to 0 which is not true. Removing these samples should help.

In [ ]:
df = df.iloc[33:] # Because of moving averages and MACD line
df = df[:-1]      # Because of shifting close price

df.index = range(len(df))
df

## Training (70%), validation (15%) and test (15%) sets

In [ ]:
test_size  = 0.15
valid_size = 0.15

test_split_idx  = int(df.shape[0] * (1-test_size))
valid_split_idx = int(df.shape[0] * (1-(valid_size+test_size)))

train_df = df.loc[:valid_split_idx].copy()
valid_df = df.loc[valid_split_idx+1:test_split_idx].copy()
test_df  = df.loc[test_split_idx+1:].copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.Date, y=train_df.Close, name='Training'))
fig.add_trace(go.Scatter(x=valid_df.Date, y=valid_df.Close, name='Validation'))
fig.add_trace(go.Scatter(x=test_df.Date,  y=test_df.Close,  name='Test'))
fig.show()

### Drop unnecessary columns

In [ ]:
refer_df = df.copy()
drop_cols = ['Date', 'Volume', 'Low', 'High', '# Deals','Value (R)', 'Move (%)', 'DY', 'EY', 'PE']

train_df = train_df.drop(drop_cols, axis=1)
valid_df = valid_df.drop(drop_cols, axis=1)
test_df  = test_df.drop(drop_cols, axis=1)


# Data Labelling and Training

In [ ]:
def train_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model_name, epochs=50, batch=34):
    nn_model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    nn_model.compile(optimizer='adam', loss=MeanAbsoluteError())

    # Train the model
    nn_model.fit(
        X_train, 
        y_train,
        epochs=epochs, 
        batch_size=batch,
        validation_data=(X_valid, y_valid)
    )

    model_checkpoint = ModelCheckpoint(f'{model_name}.h5.keras', monitor='val_loss', save_best_only=True)

    prices = df.loc[test_split_idx+1:].copy()

    y_pred = nn_model.predict(X_test)

    print(f'y_true = {np.array(y_test)[:5]}')
    print(f'y_pred = {y_pred[:5]}')
    print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')

    plt.figure(figsize=(12, 6))
    plt.plot(prices.Date, y_test, label='Actual')
    plt.plot(prices.Date, y_pred, label='Predicted')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Actual vs Predicted')
    plt.legend()
    plt.show()

    return nn_model

In [ ]:
y_train_close = train_df['Close'].copy()
X_train_close = train_df.drop(['Close'], axis=1)

y_valid_close = valid_df['Close'].copy()
X_valid_close = valid_df.drop(['Close'], axis=1)

y_test_close  = test_df['Close'].copy()
X_test_close  = test_df.drop(['Close'], axis=1)


train_model(X_train_close, y_train_close, X_valid_close, y_valid_close, X_test_close, y_test_close, 'close_model', epochs=84, batch=100).save('close_model.h5.keras')


In [ ]:
y_train_ema9 = train_df['EMA_9'].copy()
X_train_ema9 = train_df.drop(['EMA_9'], axis=1)

y_valid_ema9 = valid_df['EMA_9'].copy()
X_valid_ema9 = valid_df.drop(['EMA_9'], axis=1)

y_test_ema9  = test_df['EMA_9'].copy()
X_test_ema9  = test_df.drop(['EMA_9'] axis=1)


train_model(X_train_ema9, y_train_ema9, X_valid_ema9, y_valid_ema9, X_test_ema9, y_test_ema9, 'ema9_model').save('ema9_model.h5.keras')

In [ ]:
y_train_sma5 = train_df['SMA_5'].copy()
X_train_sma5 = train_df.drop(['SMA_5'], axis=1)

y_valid_sma5 = valid_df['SMA_5'].copy()
X_valid_sma5 = valid_df.drop(['SMA_5'], axis=1)

y_test_sma5  = test_df['SMA_5'].copy()
X_test_sma5  = test_df.drop(['SMA_5'], axis=1)

train_model(X_train_sma5, y_train_sma5, X_valid_sma5, y_valid_sma5, X_test_sma5, y_test_sma5, 'sma5_model').save('sma5_model.h5.keras')


In [ ]:
X_train_sma10 = train_df.drop(['SMA_10'], axis=1)
y_train_sma10 = train_df['SMA_10'].copy()

X_valid_sma10 = valid_df.drop(['SMA_10'], axis=1)
y_valid_sma10 = valid_df['SMA_10'].copy()

X_test_sma10  = test_df.drop(['SMA_10'], axis=1)
y_test_sma10  = test_df['SMA_10'].copy()


train_model(X_train_sma10, y_train_sma10, X_valid_sma10, y_valid_sma10, X_test_sma10, y_test_sma10, 'sma10_model').save('sma10_model.h5.keras')


In [ ]:
X_train_sma15 = train_df.drop(['SMA_15'], axis=1)
y_train_sma15 = train_df['SMA_15'].copy()

X_valid_sma15 = valid_df.drop(['SMA_15'], axis=1)
y_valid_sma15 = valid_df['SMA_15'].copy()

X_test_sma15  = test_df.drop(['SMA_15'], axis=1)
y_test_sma15  = test_df['SMA_15'].copy()

train_model(X_train_sma15, y_train_sma15, X_valid_sma15, y_valid_sma15, X_test_sma15, y_test_sma15, 'sma15_model').save('sma15_model.h5.keras')


In [ ]:
X_train_sma30 = train_df.drop(['SMA_30'], axis=1)
y_train_sma30 = train_df['SMA_30'].copy()

X_valid_sma30 = valid_df.drop(['SMA_30'], axis=1)
y_valid_sma30 = valid_df['SMA_30'].copy()

X_test_sma30  = test_df.drop(['SMA_30'], axis=1)
y_test_sma30  = test_df['SMA_30'].copy()

train_model(X_train_sma30, y_train_sma30, X_valid_sma30, y_valid_sma30, X_test_sma30, y_test_sma30, 'sma30_model').save('sma30_model.h5.keras')


In [ ]:
test_df['RSI']

In [ ]:
X_train_rsi = train_df.drop(['RSI'], axis=1)
y_train_rsi = train_df['RSI'].copy()

X_valid_rsi = valid_df.drop(['RSI'], axis=1)
y_valid_rsi = valid_df['RSI'].copy()

X_test_rsi  = test_df.drop(['RSI'], axis=1)
y_test_rsi  = test_df['RSI'].copy()

train_model(X_train_rsi, y_train_rsi, X_valid_rsi, y_valid_rsi, X_test_rsi, y_test_rsi, 'rsi_model').save('rsi_model.h5.keras')


In [ ]:
X_train_macd = train_df.drop(['MACD'], axis=1)
y_train_macd = train_df['MACD'].copy()

X_valid_macd = valid_df.drop(['MACD'], axis=1)
y_valid_macd = valid_df['MACD'].copy()

X_test_macd  = test_df.drop(['MACD'], axis=1)
y_test_macd  = test_df['MACD'].copy()

train_model(X_train_macd, y_train_macd, X_valid_macd, y_valid_macd, X_test_macd, y_test_macd, 'macd_model').save('macd_model.h5.keras')


In [ ]:
X_train_macd_signal = train_df.drop(['MACD_signal'], axis=1)
y_train_macd_signal = train_df['MACD_signal'].copy()

X_valid_macd_signal = valid_df.drop(['MACD_signal'], axis=1)
y_valid_macd_signal = valid_df['MACD_signal'].copy()

y_test_macd_signal  = test_df['MACD_signal'].copy()
X_test_macd_signal  = test_df.drop(['MACD_signal'], axis=1)

train_model(X_train_macd_signal, y_train_macd_signal, X_valid_macd_signal, y_valid_macd_signal, X_test_macd_signal, y_test_macd_signal, 'macd_signal_model').save('macd_signal_model.h5.keras')


# Fore

In [ ]:
# Load models
loaded_model_close = load_model(r'D:\Trading\notebooks\close_model.h5.keras')
loaded_model_rsi = load_model(r'D:\Trading\notebooks\rsi_model.h5.keras')
loaded_model_macd = load_model(r'D:\Trading\notebooks\macd_model.h5.keras')
loaded_model_macd_signal = load_model(r'D:\Trading\notebooks\macd_signal_model.h5.keras')

# Define models dictionary without the rolling averages
models = {
    'Close': loaded_model_close,
    'RSI': loaded_model_rsi,
    'MACD': loaded_model_macd,
    'MACD_signal': loaded_model_macd_signal
}

predictions = []

# Last known record as a dictionary (without rolling averages)
last_known = {
    'Close': 42700.0,
    'EMA_9': 43656.666091,
    'SMA_5': 43947.0,
    'SMA_10': 44178.0,
    'SMA_15': 43985.2,
    'SMA_30': 43244.2,
    'RSI': 42.109777,
    'MACD': 231.696391,
    'MACD_signal': 388.730677
}

In [ ]:
X_pred_forecast = test_df.drop('Close', axis=1)
X_pred_forecast = X_pred_forecast.iloc[-1]
X_pred_forecast

In [419]:
y_pred_forecast = loaded_model_close.predict(X_test_close)
y_pred_forecast

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[41092.473],
       [40999.902],
       [40788.242],
       [40634.875],
       [40704.977],
       [40631.27 ],
       [40517.16 ],
       [40694.945],
       [40923.145],
       [40808.09 ],
       [40807.81 ],
       [40977.48 ],
       [41119.477],
       [41130.42 ],
       [41436.08 ],
       [41729.527],
       [41950.125],
       [42030.81 ],
       [42005.844],
       [41908.215],
       [41833.875],
       [41716.29 ],
       [41727.797],
       [41822.86 ],
       [42066.164],
       [42433.36 ],
       [42816.94 ],
       [43242.492],
       [43600.605],
       [43956.9  ],
       [44222.914],
       [44359.047],
       [44351.5  ],
       [44232.777],
       [44254.29 ],
       [44310.098],
       [44317.625],
       [44477.113],
       [44920.4  ],
       [45554.4  ],
       [46017.473],
       [46440.96 ],
       [46745.484],
       [46741.945],
       [46378.094],
       [46009.96 ],
       [45609.637],
       [45217.1  ],
       [44857.395],
       [44616.734],


In [420]:
import datetime

date = datetime.datetime.strptime('2024-08-08', '%Y-%m-%d')
pd_date = pd.DataFrame([pd.to_datetime(date, format='%Y-%m-%d')])

new_row = pd.DataFrame([{
    'Date': pd_date,
    'Close': y_pred_forecast[0][0]  # Assuming prediction is an array-like structure
}])
forecasted_df = pd.concat([refer_df, new_row], ignore_index=True)
forecasted_df.tail()

,Date,Close,High,Low,Volume,# Deals,Value (R),Move (%),DY,EY,PE,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,RSI,MACD,MACD_signal
1114,2024-08-02 00:00:00,43264.000000,44943.0,43742.0,537747.0,6362.0,"236,568,075",-1.45%,1.8746,5.1748,19.3240,43784.230577,44543.2,44352.8,43803.466667,43157.666667,58.565802,513.398435,470.589155
1115,2024-08-05 00:00:00,43011.000000,43640.0,42552.0,516242.0,6469.0,"222,540,998",-1.09%,1.8953,5.2320,19.1129,43780.007519,44430.0,44365.0,43913.533333,43213.366667,55.975533,410.748740,458.621072
1116,2024-08-06 00:00:00,43164.000000,43569.0,42527.0,644856.0,10656.0,"277,224,180",-0.58%,1.9064,5.2628,19.0011,43728.406767,44227.2,44281.4,43951.266667,43237.066667,53.993949,305.461956,427.989249
1117,2024-08-07 00:00:00,42700.000000,43642.0,42848.0,854119.0,5323.0,"369,515,828",0.36%,1.8997,5.2441,19.0687,43656.666091,43947.0,44178.0,43985.200000,43244.200000,44.668850,231.696391,388.730677
1118,0 0 2024-08-08,41092.472656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
drop_cols = ['Date', 'Volume', 'Low', 'High', '# Deals','Value (R)', 'Move (%)', 'DY', 'EY', 'PE']

forecasted_df = forecasted_df.drop(drop_cols, axis=1)


In [ ]:
forecasted_df.tail()

In [ ]:
refer_df['EMA_9'] = refer_df['Close'].ewm(9).mean().shift()
refer_df['SMA_5'] = refer_df['Close'].rolling(5).mean().shift()
refer_df['SMA_10'] =refer_df['Close'].rolling(10).mean().shift()
refer_df['SMA_15'] = refer_df['Close'].rolling(15).mean().shift()
refer_df['SMA_30'] = refer_df['Close'].rolling(30).mean().shift()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date, y=df.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_5, name='SMA 5'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_10, name='SMA 10'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_15, name='SMA 15'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_30, name='SMA 30'))
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close', opacity=0.2))
fig.show()

In [ ]:
EMA_12 = pd.Series(forecasted_df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(forecasted_df['Close'].ewm(span=26, min_periods=26).mean())

forecasted_df['MACD'] = pd.Series(EMA_12 - EMA_26)
forecasted_df['MACD_signal'] = pd.Series(forecasted_df.MACD.ewm(span=9, min_periods=9).mean())

# EMA_12 = pd.Series(forecasted_df['Close'].ewm(span=12, min_periods=12).mean())
# EMA_26 = pd.Series(forecasted_df['Close'].ewm(span=26, min_periods=26).mean())

# forecasted_df['MACD'] = pd.Series(EMA_12 - EMA_26).shift()

# forecasted_df['MACD_signal'] = pd.Series(forecasted_df.MACD.ewm(span=9, min_periods=9).mean()).shift()
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_12, name='EMA 12'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_26, name='EMA 26'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=df['MACD'], name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=df['MACD_signal'], name='Signal line'), row=2, col=1)
fig.show()

In [ ]:

forecasted_df['RSI'] = relative_strength_idx(forecasted_df).shift()

forecasted_df['Close'] = forecasted_df['Close'].shift(-1)

forecast_df = forecasted_df.iloc[33:] # Because of moving averages and MACD line
forecast_df = forecast_df[:-1]  

forecasted_df.index = range(len(forecasted_df))
forecasted_df

In [ ]:

forecasted_df['EMA_9'] = forecasted_df['Close'].ewm(9).mean().shift()
forecasted_df['SMA_5'] = forecasted_df['Close'].rolling(5).mean().shift()
forecasted_df['SMA_10'] =forecasted_df['Close'].rolling(10).mean().shift()
forecasted_df['SMA_15'] = forecasted_df['Close'].rolling(15).mean().shift()
forecasted_df['SMA_30'] = forecasted_df['Close'].rolling(30).mean().shift()


EMA_12 = pd.Series(forecasted_df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(forecasted_df['Close'].ewm(span=26, min_periods=26).mean())

forecasted_df['MACD'] = pd.Series(EMA_12 - EMA_26)
forecasted_df['MACD_signal'] = pd.Series(forecasted_df.MACD.ewm(span=9, min_periods=9).mean())

forecasted_df


In [ ]:
forecasted_df['Close'] = forecasted_df['Close'].shift(-1)
forecasted_df = forecasted_df.iloc[33:] # Because of moving averages and MACD line
forecasted_df = forecasted_df[:-1]      # Because of shifting close price

forecasted_df.index = range(len(forecasted_df))
forecasted_df

In [ ]:
dates_manufacture = pd.date_range(start='2024-08-18', periods=31, freq='D')
dates = pd.DataFrame({'Date': dates_manufacture})
dates
forecasted_df = pd.concat([forecasted_df, dates], ignore_index=True)

# Reset index
forecasted_df.index = range(len(forecasted_df))


forecasted_df


# Forecasting

In [ ]:
drop_cols = ['Date', 'Volume', 'Low', 'High', '# Deals','Value (R)', 'Move (%)', 'DY', 'EY', 'PE']

starter = df.iloc[-1].drop(drop_cols)

# Load models in
loaded_model_close = load_model(r'D:\Trading\notebooks\close_model.h5.keras')
loaded_model_ema9 = load_model(r'D:\Trading\notebooks\ema9_model.h5.keras')
loaded_model_sma30 = load_model(r'D:\Trading\notebooks\sma30_model.h5.keras')

loaded_model_sma15 = load_model(r'D:\Trading\notebooks\sma15_model.h5.keras')
loaded_model_sma10 = load_model(r'D:\Trading\notebooks\sma10_model.h5.keras')
loaded_model_sma5 = load_model(r'D:\Trading\notebooks\sma5_model.h5.keras')

loaded_model_rsi = load_model(r'D:\Trading\notebooks\rsi_model.h5.keras')
loaded_model_macd = load_model(r'D:\Trading\notebooks\macd_model.h5.keras')
loaded_model_macd_signal= load_model(r'D:\Trading\notebooks\macd_signal_model.h5.keras')

# Assuming you have trained models stored in a dictionary
models = {
    'Close': loaded_model_close,
    # 'EMA_9': loaded_model_ema9,
    # 'SMA_5': loaded_model_sma5,
    # 'SMA_10': loaded_model_sma10,
    # 'SMA_15': loaded_model_sma15,
    # 'SMA_30': loaded_model_sma30,
    # 'RSI': loaded_model_rsi,
    # 'MACD': loaded_model_macd,
    # 'MACD_signal': loaded_model_macd_signal
}

predictions = []

# Last known record as a dictionary
last_known = {
    'Close': 42700.0,
    'EMA_9': 43656.666091,
    'SMA_5': 43947.0,
    'SMA_10': 44178.0,
    'SMA_15': 43985.2,
    'SMA_30': 43244.2,
    'RSI': 42.109777,
    'MACD': 231.696391,
    'MACD_signal': 388.730677
}
next_day = {}


def model_predict(model, feature, X_test):
    next_day[feature] = model.predict(input_data)[0, 0] # Adjusting to ensure scalar output
    
    return next_day
    

for feature, model in models.items():
    # Convert the current dictionary to a DataFrame for prediction
    input_data = pd.DataFrame([last_known]).drop(feature, axis=1)

    # Predict the next day's value for the feature
    # next_day[feature] = model.predict(input_data)[0]
    next_day = model_predict(model, feature, input_data)


# Append the predictions to the list
predictions.append(next_day)

# Update last_known with the next day's predictions
last_known = next_day


In [ ]:


# for day in range(30):
#     next_day = {}

#     # Step 1: Predict the 'Close' price first
#     input_data_close = pd.DataFrame([last_known])
#     next_day['Close'] = models['Close'].predict(input_data_close)

#     # Step 2: Recalculate rolling averages based on the predicted 'Close' price
#     df_temp = pd.concat([input_data_close, pd.DataFrame([{'Close': next_day['Close']}])], ignore_index=True)

#     df_temp['EMA_9'] = df_temp['Close'].ewm(9).mean()
#     df_temp['SMA_5'] = df_temp['Close'].rolling(5).mean()
#     df_temp['SMA_10'] = df_temp['Close'].rolling(10).mean()
#     df_temp['SMA_15'] = df_temp['Close'].rolling(15).mean()
#     df_temp['SMA_30'] = df_temp['Close'].rolling(30).mean()

#     last_known['EMA_9'] = df_temp['EMA_9']
#     last_known['SMA_5'] = df_temp['SMA_5']
#     last_known['SMA_10'] = df_temp['SMA_10']
#     last_known['SMA_15'] = df_temp['SMA_15']
#     last_known['SMA_30'] = df_temp['SMA_30']

In [ ]:
drop_cols = ['Date', 'Volume', 'Low', 'High', '# Deals','Value (R)', 'Move (%)', 'DY', 'EY', 'PE']

starter = df.iloc[-1].drop(drop_cols)

# Load models
loaded_model_close = load_model(r'D:\Trading\notebooks\close_model.h5.keras')
loaded_model_rsi = load_model(r'D:\Trading\notebooks\rsi_model.h5.keras')
loaded_model_macd = load_model(r'D:\Trading\notebooks\macd_model.h5.keras')
loaded_model_macd_signal = load_model(r'D:\Trading\notebooks\macd_signal_model.h5.keras')

# Define models dictionary without the rolling averages
models = {
    'Close': loaded_model_close,
    'RSI': loaded_model_rsi,
    'MACD': loaded_model_macd,
    'MACD_signal': loaded_model_macd_signal
}

predictions = []

# Last known record as a dictionary (without rolling averages)
last_known = {
    'Close': 42700.0,
    'EMA_9': 43656.666091,
    'SMA_5': 43947.0,
    'SMA_10': 44178.0,
    'SMA_15': 43985.2,
    'SMA_30': 43244.2,
    'RSI': 42.109777,
    'MACD': 231.696391,
    'MACD_signal': 388.730677
}

for day in range(30):
    next_day = {}

    # Step 1: Predict the 'Close' price first
    input_data_close = pd.DataFrame([last_known])
    next_day['Close'] = models['Close'].predict(input_data_close)

    # Step 2: Recalculate rolling averages based on the predicted 'Close' price

    df_temp['EMA_9'] = df_temp['Close'].ewm(9).mean()
    df_temp['SMA_5'] = df_temp['Close'].rolling(5).mean()
    df_temp['SMA_10'] = df_temp['Close'].rolling(10).mean()
    df_temp['SMA_15'] = df_temp['Close'].rolling(15).mean()
    df_temp['SMA_30'] = df_temp['Close'].rolling(30).mean()

    last_known['EMA_9'] = df_temp['EMA_9']
    last_known['SMA_5'] = df_temp['SMA_5']
    last_known['SMA_10'] = df_temp['SMA_10']
    last_known['SMA_15'] = df_temp['SMA_15']
    last_known['SMA_30'] = df_temp['SMA_30']

    # # Step 3: Use recalculated rolling averages for predicting other features
    # for feature in models.keys():
    #     if feature == 'Close':
    #         continue
    #     input_data = pd.DataFrame([last_known]).drop(feature, axis=1)
    #     next_day[feature] = models[feature].predict(input_data)[0, 0]

    # Append the predictions to the list
    predictions.append(next_day)

    # Update last_known with the next day's predictions
    last_known.update(next_day)


In [ ]:
predictions_df = pd.DataFrame(predictions)
dates = pd.date_range(start='2024-08-15', periods=30, freq='D')
predictions_df['Date'] = dates
plt.figure(figsize=(12, 6))
import seaborn as sns
sns.lineplot(x=predictions_df['Date'], y=predictions_df['Close'], label='Predicted', color='red')


In [ ]:
drop_cols = ['Date', 'Volume', 'Low', 'High', '# Deals','Value (R)', 'Move (%)', 'DY', 'EY', 'PE']

starter = df.iloc[-1].drop(drop_cols)

# Load models in
loaded_model_close = load_model(r'D:\Trading\notebooks\close_model.h5.keras')
loaded_model_ema9 = load_model(r'D:\Trading\notebooks\ema9_model.h5.keras')
loaded_model_sma30 = load_model(r'D:\Trading\notebooks\sma30_model.h5.keras')

loaded_model_sma15 = load_model(r'D:\Trading\notebooks\sma15_model.h5.keras')
loaded_model_sma10 = load_model(r'D:\Trading\notebooks\sma10_model.h5.keras')
loaded_model_sma5 = load_model(r'D:\Trading\notebooks\sma5_model.h5.keras')

loaded_model_rsi = load_model(r'D:\Trading\notebooks\rsi_model.h5.keras')
loaded_model_macd = load_model(r'D:\Trading\notebooks\macd_model.h5.keras')
loaded_model_macd_signal= load_model(r'D:\Trading\notebooks\macd_signal_model.h5.keras')

# Assuming you have trained models stored in a dictionary
models = {
    'Close': loaded_model_close,
    'EMA_9': loaded_model_ema9,
    'SMA_5': loaded_model_sma5,
    'SMA_10': loaded_model_sma10,
    'SMA_15': loaded_model_sma15,
    'SMA_30': loaded_model_sma30,
    'RSI': loaded_model_rsi,
    'MACD': loaded_model_macd,
    'MACD_signal': loaded_model_macd_signal
}

predictions = []

# Last known record as a dictionary
last_known = {
    'Close': 42700.0,
    'EMA_9': 43656.666091,
    'SMA_5': 43947.0,
    'SMA_10': 44178.0,
    'SMA_15': 43985.2,
    'SMA_30': 43244.2,
    'RSI': 42.109777,
    'MACD': 231.696391,
    'MACD_signal': 388.730677
}

for day in range(30):
    next_day = {}
    for feature, model in models.items():
        # Convert the current dictionary to a DataFrame for prediction
        input_data = pd.DataFrame([last_known]).drop(feature, axis=1)

        # Predict the next day's value for the feature
        # next_day[feature] = model.predict(input_data)[0]
        next_day[feature] = model.predict(input_data)[0, 0]  # Adjusting to ensure scalar output

    # Append the predictions to the list
    predictions.append(next_day)

    # Update last_known with the next day's predictions
    last_known = next_day
